In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mnist-dataset/train-images.idx3-ubyte
/kaggle/input/mnist-dataset/t10k-labels.idx1-ubyte
/kaggle/input/mnist-dataset/t10k-images.idx3-ubyte
/kaggle/input/mnist-dataset/train-labels.idx1-ubyte
/kaggle/input/mnist-dataset/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte
/kaggle/input/mnist-dataset/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte
/kaggle/input/mnist-dataset/train-labels-idx1-ubyte/train-labels-idx1-ubyte
/kaggle/input/mnist-dataset/train-images-idx3-ubyte/train-images-idx3-ubyte


In [8]:
# Import all the liberaries 
import torch 
import torch.nn as nn
import tensorflow as tf
import pandas as pd
from torch.utils.data import DataLoader , Dataset

In [9]:
# Import minist dataset
(x_train ,y_train ),(x_test  , y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000 , 784)
x_test = x_test.reshape(10000 , 784)
x_train.shape , y_train.shape , x_test.shape , y_test.shape , y_train.shape


((60000, 784), (60000,), (10000, 784), (10000,), (60000,))

In [10]:
# check for GPUs
torch.cuda.is_available()

# make gpu as device
device = torch.device("cuda")
print(f"using device {device}")

using device cuda


In [11]:
# Converting into Tensors for Pytorch

x_train = np.array(x_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
y_train = np.array(y_train)

x_train = torch.from_numpy(x_train)
x_test = torch.from_numpy(x_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)


In [12]:
# Create model class
import torch.optim as opt

class NeuralNetwork(nn.Module):
    def __init__(self , feature_count):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(feature_count , 256 , bias= True),
            nn.ReLU(),
            nn.Linear(256 , 128 , bias= True),
            nn.ReLU(),
            nn.Linear(128 , 64 ),
            nn.ReLU(),
            nn.Linear(64 , 10),
        )
        
    def forward(self , features):
        out = self.network(features)
        return out
    


In [13]:
# Create Dataset class
class CustomeDataset(Dataset):
    def __init__(self, featues , lables):
        
        self.features = featues.to(dtype = torch.float32)
        self.features = self.features/255
        self.labels = lables.to(dtype = torch.long)
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self , index):
        return self.features[index] , self.labels[index]


In [14]:
# Declare hyperparamater 
batch_size = 16
lr = 0.01
epochs = 20

In [15]:
# Init model and Dataloder 
dataset_train = CustomeDataset(x_train , y_train)
dataloder_train = DataLoader(dataset_train , shuffle= True , batch_size= batch_size)

# Model
model = NeuralNetwork(x_train.shape[1])
model = model.to(device)

loss_function = nn.CrossEntropyLoss()
opti = opt.Adam(model.parameters() , lr = lr)

In [16]:
x_train.shape

torch.Size([60000, 784])

In [17]:
from torchinfo import summary
summary(model , input_size=(60000, 784))

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [60000, 10]               --
├─Sequential: 1-1                        [60000, 10]               --
│    └─Linear: 2-1                       [60000, 256]              200,960
│    └─ReLU: 2-2                         [60000, 256]              --
│    └─Linear: 2-3                       [60000, 128]              32,896
│    └─ReLU: 2-4                         [60000, 128]              --
│    └─Linear: 2-5                       [60000, 64]               8,256
│    └─ReLU: 2-6                         [60000, 64]               --
│    └─Linear: 2-7                       [60000, 10]               650
Total params: 242,762
Trainable params: 242,762
Non-trainable params: 0
Total mult-adds (G): 14.57
Input size (MB): 188.16
Forward/backward pass size (MB): 219.84
Params size (MB): 0.97
Estimated Total Size (MB): 408.97

In [18]:
# traing Loop

for epoch in range(epochs):
    total_epoch_loss = 0
    
    for train_features , train_lables in dataloder_train:

        # move item to GPUs
        train_features , train_lables = train_features.to(device) , train_lables.to(device)
        # forward pass
        y_pred = model(train_features)
        
        # Loss calculation
        loss = loss_function(y_pred , train_lables)
        
        # clear gradient 
        opti.zero_grad()
         
        # backpropgation
        loss.backward()
        
        # paramter updates
        opti.step()
        
        total_epoch_loss += loss.item()
    
    print(f"Epoch : {epoch+1} , Loss : {total_epoch_loss/len(dataloder_train)}")
    

Epoch : 1 , Loss : 0.3554515165738141
Epoch : 2 , Loss : 0.24260682254629987
Epoch : 3 , Loss : 0.23722406359018836
Epoch : 4 , Loss : 0.1915044701123552
Epoch : 5 , Loss : 0.18471293000269118
Epoch : 6 , Loss : 0.1733962741867855
Epoch : 7 , Loss : 0.1615028805149929
Epoch : 8 , Loss : 0.1699823418313102
Epoch : 9 , Loss : 0.16172361695969073
Epoch : 10 , Loss : 0.15858902210206038
Epoch : 11 , Loss : 0.16851900136497358
Epoch : 12 , Loss : 0.16051397308745297
Epoch : 13 , Loss : 0.13680093674108887
Epoch : 14 , Loss : 0.13665338478701564
Epoch : 15 , Loss : 0.1456899383184439
Epoch : 16 , Loss : 0.13791604651889602
Epoch : 17 , Loss : 0.13155803647550882
Epoch : 18 , Loss : 0.19048354887858668
Epoch : 19 , Loss : 0.16863729645364123
Epoch : 20 , Loss : 0.1843239981772095


In [22]:
# Evalution 

model.eval()
test_dataset = CustomeDataset(x_test , y_test)
test_dataloder = DataLoader(test_dataset , batch_size= 16 , shuffle= False)

total = 0
correct = 0

with torch.no_grad():
    for test_feat , test_label in dataloder_train:

        # move data to GPUs
        test_feat , test_label = test_feat.to(device) , test_label.to(device)
        
        output = model(test_feat)
        
        _ , pos = torch.max(output , dim= 1)
        
        total += test_label.shape[0]
        
        correct += (pos == test_label).sum().item()
    
    print(f"Tesing Accuracy : {correct/total}")
        
        

Tesing Accuracy : 0.9467333333333333


In [23]:
# Evalution 

model.eval()
test_dataset = CustomeDataset(x_test , y_test)
test_dataloder = DataLoader(test_dataset , batch_size= 16 , shuffle= False)

total = 0
correct = 0

with torch.no_grad():
    for test_feat , test_label in test_dataloder:

        # move data to GPUs
        test_feat , test_label = test_feat.to(device) , test_label.to(device)
        
        output = model(test_feat)
        
        _ , pos = torch.max(output , dim= 1)
        
        total += test_label.shape[0]
        
        correct += (pos == test_label).sum().item()
    
    print(f"Training Accuracy : {correct/total}")
        
        

Training Accuracy : 0.9349
